In [1]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import os

In [2]:
df = pd.read_csv("../data/cleaned_for_generation.csv")

In [3]:
df = df[["input", "output"]].fillna("").astype(str)
qa_pairs = df.to_dict(orient="records")  # قائمة من القواميس [{'input': ..., 'output': ...}, ...]


In [4]:
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)

questions = [pair["input"] for pair in qa_pairs]
embeddings = model.encode(questions, show_progress_bar=True, batch_size=64)

C:\Users\DELL\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/12633 [00:00<?, ?it/s]

In [7]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype("float32"))

In [8]:
os.makedirs("../embeddings", exist_ok=True)
faiss.write_index(index, "../embeddings/faiss_index.bin")
np.save("../embeddings/medical_questions.npy", qa_pairs, allow_pickle=True)

print(" تم بناء فهرس FAISS وحفظ الأسئلة + الإجابات.")

 تم بناء فهرس FAISS وحفظ الأسئلة + الإجابات.
